# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [6]:
import os
import re
import random
from collections import defaultdict, Counter
from itertools import permutations

def load_ngrams(file_path):
    ngram_dict = {}
    with open(file_path, "r", encoding="latin-1") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) > 1:
                freq = int(parts[0])
                ngram = tuple(parts[1:])
                ngram_dict[ngram] = freq
    return ngram_dict

data_files = ["bigrams (2).txt"]
ngram_data = {file: load_ngrams(file) for file in data_files}

bigrams = ngram_data["bigrams (2).txt"]

def extract_unigrams_from_ngrams(bigrams):
    unigrams = defaultdict(int)
    for (w1, w2), freq in bigrams.items():
        unigrams[w1] += freq
        unigrams[w2] += freq
    return unigrams

unigrams = extract_unigrams_from_ngrams(bigrams)

def damerau_levenshtein(s1, s2):
    d = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]
    for i in range(len(s1) + 1):
        for j in range(len(s2) + 1):
            if i == 0:
                d[i][j] = j
            elif j == 0:
                d[i][j] = i
            else:
                cost = 0 if s1[i - 1] == s2[j - 1] else 1
                d[i][j] = min(d[i - 1][j] + 1,
                               d[i][j - 1] + 1,
                               d[i - 1][j - 1] + cost)
                if (i > 1 and j > 1 and s1[i - 1] == s2[j - 2] and s1[i - 2] == s2[j - 1]):
                    d[i][j] = min(d[i][j], d[i - 2][j - 2] + 1)
    return d[len(s1)][len(s2)]

def generate_candidates(word):
    candidates = {word}
    for unigram in unigrams.keys():
        if abs(len(word) - len(unigram)) <= 2 and damerau_levenshtein(word, unigram) <= 2:
            candidates.add(unigram)
    return candidates if candidates else {word}

class SpellCorrector:
    def __init__(self, unigrams, bigrams, beam_width):
        self.unigrams = unigrams
        self.bigrams = bigrams
        self.total_unigrams = sum(unigrams.values())
        self.beam_width = beam_width

    def P(self, word):
        return self.unigrams.get(word, 1) / self.total_unigrams

    def P_bigram(self, w1, w2):
        return self.bigrams.get((w1, w2), 1) / self.unigrams.get(w1, 1)

    def correct(self, sentence):
        words = sentence.lower().split()
        corrected_sentence = []
        
        for idx, word in enumerate(words):
            candidates = generate_candidates(word)
            if idx == 0:
                best_word = max(candidates, key=lambda w: self.P(w))
            else:
                best_word = max(candidates, key=lambda w: self.P(w) * self.P_bigram(corrected_sentence[-1], w))
            corrected_sentence.append(best_word)
        
        return " ".join(corrected_sentence)


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

Why Use Bigrams?
Bigrams capture local context, meaning they help correct misspelled words based on their immediate neighbors. This is particularly useful in distinguishing between words that are often confused based on surrounding text. For example, in the phrase "dking sport", the bigram model helps prioritize "doing sport" over "dying sport" because "doing sport" is more common in the corpus.

Why Not Fivegrams?
Originally, a fivegram approach was suggested for capturing longer-range dependencies. However, fivegrams require significantly larger datasets and suffer from sparsity issues. Instead, we rely on bigrams and unigrams to balance context awareness and computational efficiency.

Why Use This Approach?
Our approach combines:

Unigram frequency: Helps prioritize commonly used words.
Bigram probabilities: Captures local context to disambiguate similar-looking words.
Damerau-Levenshtein distance: Generates realistic spelling correction candidates by considering common types of errors (insertions, deletions, substitutions, and transpositions).
Challenge: Context Sensitivity
Correcting words based on both context and spelling similarity is difficult. Our solution:

Uses bigrams to guide corrections based on likely word sequences.
Uses Damerau-Levenshtein distance to suggest spelling alternatives.
Uses probability-based selection to choose the most likely correction.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [11]:
def introduce_errors(sentence: str, error_prob=0.3) -> str:
    words = sentence.split()
    noisy_words = []
    for word in words:
        if random.random() < error_prob:
            noisy_words.append(random.choice(list(generate_candidates(word)) + [word]))
        else:
            noisy_words.append(word)
    return " ".join(noisy_words)

test = {}
with open("fivegrams (2).txt", "r", encoding="latin-1") as f:
    for i, line in enumerate(f):
        if i % 100000 == 0:
            parts = line.strip().split()
            if len(parts) > 1:
                freq = int(parts[0])
                ngram = " ".join(parts[1:])
                test[ngram] = introduce_errors(ngram)

def evaluate(corrector, test_sentences):
    total, correct = 0, 0
    for k, v in test_sentences.items():
        corrected_sentence = corrector.correct(v)
        print(f"corrected: {corrected_sentence}  true: {k}  error: {v}")
        if corrected_sentence == k:
            correct += 1
        total += 1
    return correct / total

beam_width = 5
corrector = SpellCorrector(unigrams, bigrams, beam_width)

accuracy = evaluate(corrector, test)
print(f"Accuracy: {accuracy:.2%}")

corrected: of have to the old  true: a babe in the woods  error: a babe in the food
corrected: the likely to the white  true: are likely to be quite  error: are unlikely to be quite
corrected: crisis that he or the  true: crisis that led to the  error: crisis that led bol the
corrected: a new set in a  true: has been used in a  error: has hewn used sion a
corrected: of a very good position  true: in a very good position  error: my a very gto position
corrected: look at the one mile  true: looked at her and smiled  error: looked tap her and smiled
corrected: and to the door to  true: one of the four or  error: and koy the four or
corrected: all about the decline in  true: talk about the decline of  error: talk snout the decline of
corrected: the strength of the u.s  true: the strength of the u.s  error: shed strength of ihl u.s
corrected: the other findings of a  true: to the findings of a  error: to the findings of yaw
corrected: his and the children and  true: wife and two children in

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)